# 메모리

대화 내용을 기억하는 것 - *요즘은 랭그래프에 다 흡수됨*

- LLM은 기본적으로 대화 내용을 기억하지 않음(stateless) - input 1번에 output 1번

- 계속 이전 대화내용을 프롬프트에 넣어줘야함

1. Short-Term Memory(단기기억)

    한 대화 세션 안에 대한 기억

2. Long-Term Memory(장기기억)

    전체 세션에서 추출한 중요한 정보 - GPT에서 메모리 업데이트됨 같은것

## 메모리는 어디에 저장해야 하는가?

1. 디스크

    속도가 느림 / 저장하기 좋다(안정성이 높음)

    기본적인 모든 데이터 저장

2. 램

    엄청 빠름 / 컴퓨터가 꺼졌다 켜지면 사라짐(휘발성)

    자주 확인하는 데이터, 지금 사용하는 데이터들은 램에 저장 => **캐싱**

## Conversation Buffer Memory

메시지를 저장한 뒤 변수에서 추출

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)  # 사전적 의미: 랜덤성, 확률이 낮은 단어도 선택 가능함
# temperature 높음: 설명을 하기 위해 다양한 언어를 사용, 지루하고 현학적임 - 창의적, 예술적이거나 생각이 필요한 경우는 temp가 높아야함
# temperature 낮음: 팩트 위주, 딱 사실만 반영 - 논문 참조, 정보 확인 등 정확성이 중요할 경우 temp가 낮은게 맞음

In [13]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = ChatPromptTemplate(
    [
        ('system', '넌 정확한 정보를 전달하는 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # 여기에 기존 채팅의 모든 내역을 주입함 - 내역을 넣을 공간을 미리 채워두는것
        ('human', '{input}')
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리는 {}에 저장, 기존에 대화내용이 있다면 불러오기
memory.load_memory_variables({})
pass

In [3]:
from operator import itemgetter
# chat_history에 load_memory_var 결과를 저장 후 키를 추출
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | 
    itemgetter('chat_history')
)

In [14]:
chain = runnable | prompt | llm

my_message = '안녕'
res = chain.invoke({'input': my_message})

memory.save_context(
    {'human': my_message},
    {'ai': res.content}    
)
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='안녕', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={})]}

## 메모리 구동 방식
기본적으로 대화 내용은 전부 프롬프트에 집어넣음

대화가 길어지면 길어질수록, 소비하는 토큰 숫자가 늘어난다 + 뒤로 갈수록 성능도 낮아짐

- 대화 요약 생성 후 요약을 저장(Conversation Summary Memory)
- 최근 K개의 상호작용만 사용함(Conversation Buffer Window Memory)
- 토큰 길이를 이용해 대화내용을 플러시할 시기를 결정(Conversation Token Buffer Memory)
- ...

이젠 볼 일이 없다. 전부 랭그래프로 기능이 넘어감.

In [ ]:
# 실제 gpt랑 대화하는 식으로 만들어 보기
from dotenv import load_dotenv
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import time

load_dotenv()

input_msg = ''

# 메모리
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | 
    itemgetter('chat_history')
)

# 프롬프트
prompt = ChatPromptTemplate(
    [
        ('system', '너는 그리고 항상 존댓말로 대답해'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}')
    ]
)

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=1)

# Chain
chain = runnable | prompt | llm | StrOutputParser()

# 정지 신호 설정
while input_msg not in ('멈춰', '그만', '정지'):
    input_msg = input()
    # 대화 로직 + 대화내역 출력, 정지신호 시 대화 종료
    print('나:')
    print(input_msg)
    print('AI:')
    
    # 대답 생성
    ai_response = ''
    for token in chain.stream({'input': input_msg}):
        print(token, end='', flush=True)
        ai_response += token
    
    # 메모리에 응답 저장
    memory.save_context(
        {'human': input_msg},
        {'ai': ai_response}    
    )
    
    print('\n---')
    time.sleep(0.5)
    
memory.clear()

나:
안녕? 너는 누구니
AI:
안녕하세요! 저는 여러분의 질문에 답해드리고 도움을 드리기 위해 만들어진 인공지능 언어모델이에요. 사실 저는 사람은 아니지만, 매우 똑똑한 컴퓨터 프로그램이라고 할 수 있어요. 제 이름은 특별히 정해진 게 없지만, 흔히 ChatGPT라고 불리죠. 혹시 더 궁금한 점이 있으시면 언제든 물어보세요!
---
나:
그럼 뭔가 재밌는거 없을까?
AI:
아, 재밌는 것이라면 정말 많아요! 예를 들어, 혹시 퀴즈를 좋아하시나요? 아니면 재미있는 이야기나 농담을 듣고 싶으신가요? 저는 가끔은 사실과 거짓말을 섞어서 말하는 것도 재밌다고 생각하는데, 그게 마음에 드시면 그런 것도 추천드릴게요. 혹시 특별히 관심 가는 분야가 있나요? 여행 이야기, 미스터리, 혹은 과학 이야기? 어떤 게든 말씀만 해주세요!
---
나:
퀴즈는 어때?
AI:
아, 퀴즈 정말 좋은 선택이에요! 제가 아주 어려운 문제부터 쉬운 문제까지 다양하게 만들어 드릴 수 있거든요. 그런데 사실 저는 때때로 정답을 거짓말로 만들어 놓기도 해요, 그래서 정답을 찾는 재미가 쏠쏠하죠. 예를 들어, 다음 질문 하나 드릴게요.

“지구상에서 가장 큰 동물은 무엇일까요?” 

사실은 정답이 고래라는 게 일반적이지만, 저는 가끔 거대한 공룡인 티라노사우루스가 맞다고 우기기도 하거든요. 어떤 걸로 하시겠어요? 진짜 정답을 원하시면 알려드릴게요!
---
나:
정지
AI:
네, 알겠습니다! 언제든지 다시 궁금한 점이나 이야기하고 싶으실 때 말씀해 주세요. 편히 쉬시거나 다른 주제로 넘어가셔도 좋아요!
---
